# Questioning Barbie and Oppenheimer Through the Use of Agents

In the following notebook we will build an application that queries both the Barbie and Oppenheimer movies Wikipedia pages, as well as their reviews.

The main focus of this notebook is to showcase a brief introduction to Agents.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Contruct a Barbie retriever
2. Construct an Oppenheimer retriever
3. Combine the two and allow users to query both resources from a single input through the use of Agents

## Ship 🚢

Based on Tuesday's session - construct a Chainlit (or Gradio) application that allows users to interface with the application.

## Share 🚀

Make a social media post about your final application.

### Dependencies

As always, let's start with some dependencies!

In [2]:
!pip install -q -U langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00


In [1]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your OpenAI API Key: ··········


### LLM

We will be leveraging OpenAI's `gpt-3.5-turbo` throughout the notebook, and we can keep it consistent throughout!

In [17]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia.

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [4]:
!pip install -q -U wikipedia

  Preparing metadata (setup.py) ... done


In [6]:
from langchain.document_loaders import WikipediaLoader, CSVLoader

barbie_wikipedia_docs = WikipediaLoader(
    query="Barbie (film)",
    load_max_docs= 1,
    doc_content_chars_max=1000000
    ).load()

barbie_csv_docs = CSVLoader(
    file_path="./barbie.csv",
    source_column="Review_Url"
    ).load()

Since we'll be using same format source documentation separated by topic, we can save ourselves some extra effort and set up our splitters once.

We're going to leverage the `RecursiveCharacterTextSplitter` again, this time paying close attention to the format our Wikipedia articles and reviews are in so we can be sure to chunk them appropritately.

> HINT: You can pass a list of separators when you intialize your `RecursiveTextSplitter`! They are acted on in order of element 0 -> element len(list).

RELEVANT DOCS:
- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0,
    length_function = len,
    is_separator_regex= False,
    separators = ["\n==", "\n", " "]
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    length_function = len,
    is_separator_regex= False,
    separators = ["\n", " "]
)

chunked_barbie_wikipedia_docs = wikipedia_text_splitter.transform_documents(barbie_wikipedia_docs)
chunked_barbie_csv_docs = csv_text_splitter.transform_documents(barbie_csv_docs)

#### Retrieval and Embedding Strategy

We've already discussed the useful application of `CacheBackedEmbeddings`, so let's do it again!

RELEVANT DOCS:
- [`CacheBackedEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.cache.CacheBackedEmbeddings.html#langchain-embeddings-cache-cachebackedembeddings)

In [9]:
!pip install -q -U rank_bm25 tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 113.5 MB/s eta 0:00:00


In [10]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# set up cached embeddings store
store = LocalFileStore("./cache/")

core_embeddings_model = OpenAIEmbeddings()

embedder = CacheBackedEmbeddings.from_bytes_store(core_embeddings_model, store, namespace=core_embeddings_model.model)


We'll implement a `FAISS` vectorstore, and create a retriever from it.

In [11]:
barbie_csv_faiss_retriever = FAISS.from_documents(chunked_barbie_csv_docs, embedder).as_retriever()


There are a number of excellent options to retrieve documents - we'll be looking at an additional example today, which is called the `EnsembleRetriever`.

The method this is using is outlined in [this paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

The brief explanation is:

1. We collect results from two different retrieval methods over the same corpus
2. We apply a reranking algorithm to rerank our source documents to be the *most relevant* without losing specific or potentially low-ranked information rich documents
3. We feed the top-k results into the LLM with our query as context.

> HINT: Your weight list should be of type `List[float]` and the `sum(List[float])` should be `1`.

In [12]:
# set up BM25 retriever
barbie_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_barbie_wikipedia_docs
)
barbie_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
barbie_wikipedia_faiss_store = FAISS.from_documents(
    chunked_barbie_wikipedia_docs,
    embedder
)
barbie_wikipedia_faiss_retriever = barbie_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
barbie_ensemble_retriever = EnsembleRetriever(
    retrievers=[barbie_wikipedia_bm25_retriever, barbie_wikipedia_faiss_retriever],
    weights=[0.25, 0.75]
)

#### Retrieval Agent

We can create a simple conversational retrieval Agent by using the built-ins provided by LangChain!

> HINT: Be sure to provide good natural language descriptions of what the tool should be used for to get the best results.

RELEVANT DOCS:
- [`create_retriever_tool`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool)

In [13]:
from langchain.agents.agent_toolkits import create_retriever_tool

barbie_wikipedia_retrieval_tool = create_retriever_tool(
    barbie_ensemble_retriever,
    "Wikipedia",
    "Searches and returns information about the Barbie movie"
)

barbie_csv_retrieval_tool = create_retriever_tool(
    barbie_csv_faiss_retriever,
    "Reviews",
    "Searches and returns reviews written about the Barbie movie"
)

barbie_retriever_tools = [barbie_wikipedia_retrieval_tool, barbie_csv_retrieval_tool]

Now that we've created our tools, we can combined them into an agent!

RELEVANT DOCS:
- [`create_conversational_retrieval_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent.html#langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent)

In [18]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

barbie_retriever_agent_executor = create_conversational_retrieval_agent(llm, barbie_retriever_tools, verbose=True)

In [19]:
barbie_retriever_agent_executor({"input" : "Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?"})



> Entering new AgentExecutor chain...

Invoking: `Reviews` with `Barbie movie`


[Document(page_content="Review: The big summer blockbuster of 2023 is a wonderful visual spectacle with a great deal of creative physical comedy, centered in and around the universe of Barbie, Mattel's historic toy doll. Unfortunately, the film tries to be and do much more than what the filmmakers ultimately are able to control and reel in. That a movie about Barbie would be guilty of Americanization is no surprise, and in many ways more than befitting, but the Americanization here isn't just about the history of the toy itself and all its cultural impacts - Barbie also views everything through the eyes of Hollywood anno 2023 - which arguably is a very particular and perhaps not that representative outlook. Instead of utilizing the fine foundation laid in the film's first third, director and co-writer Greta Gervig (Lady Bird, Little Women) gets herself lost in a trite, divisive and most likely futile bat

{'input': 'Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'Reviews', 'arguments': '{\n  "__arg1": "Barbie movie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content="Review: The big summer blockbuster of 2023 is a wonderful visual spectacle with a great deal of creative physical comedy, centered in and around the universe of Barbie, Mattel\'s historic toy doll. Unfortunately, the film tries to be and do much more than what the filmmakers ultimately are able to control and reel in. That a movie about Barbie would be guilty of Americanization is no surprise, and in many ways more than befitting, but the Americanizatio

In [20]:
barbie_retriever_agent_executor({"input" : "What is a very quick summary of the plot of the Barbie movie?"})



> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `Barbie movie`


[Document(page_content='Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self-discovery following an existential crisis. It features an ensemble supporting cast, including America Ferrera, Kate McKinnon, Helen Mirren, Issa Rae, Simu Liu,', metadata={'title': 'Barbie (film)', 'summary': 'Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self

{'input': 'What is a very quick summary of the plot of the Barbie movie?',
 'chat_history': [AIMessage(content="Based on the reviews of the Barbie movie, opinions are mixed. Some people enjoyed the movie and found it entertaining, while others had some criticisms.\n\nOne reviewer mentioned that the movie is a wonderful visual spectacle with creative physical comedy. However, they felt that the film tried to be and do more than what the filmmakers were able to control. They mentioned that the movie views everything through the eyes of Hollywood in 2023, which may not be representative for everyone. The reviewer also mentioned that the film gets lost in a trite and divisive battle of the sexes, which could leave some viewers feeling confused.\n\nAnother reviewer rated the movie as good but not great. They mentioned that the film touched many people, with some enjoying the fun of seeing a doll come to life and others appreciating the theme of empowerment. However, they noted that the movi

### Oppenheimer Retrieval System

We're going to repourpose some of what we created previously, but this time we'll explore a different multi-source retrieval system.

In [21]:
oppenheimer_wikipedia_docs = WikipediaLoader(query="Oppenheimer (film)", load_max_docs=1, doc_content_chars_max=1_000_000).load()
oppenheimer_csv_docs = CSVLoader(file_path="./oppenheimer.csv", source_column="Review_Url").load()

In [22]:
chunked_opp_wikipedia_docs = wikipedia_text_splitter.transform_documents(oppenheimer_wikipedia_docs)
chunked_opp_csv_docs = csv_text_splitter.transform_documents(oppenheimer_csv_docs)

In [23]:
opp_csv_faiss_retriever = FAISS.from_documents(chunked_opp_csv_docs, embedder).as_retriever()

# set up BM25 retriever
opp_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_opp_wikipedia_docs
)
opp_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
opp_wikipedia_faiss_store = FAISS.from_documents(
    chunked_opp_wikipedia_docs, embedder
)
opp_wikipedia_faiss_retriever = opp_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
opp_ensemble_retriever = opp_ensemble_retriever = EnsembleRetriever(
    retrievers=[opp_wikipedia_bm25_retriever, opp_wikipedia_faiss_retriever],
    weights=[0.25, 0.75] # should sum to 1
)

#### Multi-source chain

We're going to allow the LLM to decide which information is most -> least valuable.

The way we'll do this is with LangChain's rather powerful "Expression Language"!

> HINT: You can leverage [this](https://python.langchain.com/docs/use_cases/question_answering/how_to/multiple_retrieval) resource if you get stuck - but experiment with different prompts/formats.

In [24]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Oppenheimer movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Oppenheimer movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])

In [25]:
oppenheimer_multisource_chain = {
    "source1": (lambda x: x["question"]) | opp_ensemble_retriever,
    "source2": (lambda x: x["question"]) | opp_csv_faiss_retriever,
    "question": lambda x: x["question"],
} | prompt | llm

In [26]:
oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

AIMessage(content="People generally thought highly of the Oppenheimer movie. Critics praised the screenplay, performances of the cast (especially Cillian Murphy and Robert Downey Jr.), and the visuals. It was frequently cited as one of the best films of 2023. On the review aggregator website Rotten Tomatoes, 93% of 451 critics' reviews were positive, with an average rating of 8.6/10. The movie received critical acclaim, with particular praise for the cast performances, screenplay, and visuals. Some criticism was aimed towards the writing of the female characters. Overall, audiences found the movie compelling, engaging, and simple to understand.", additional_kwargs={}, example=False)

# Agent Creation

Now we can finally start building our Agent!

The first thing we'll need to do is provide our Agent a Toolbelt. (list of tools). Much like Batman, our LLM-powered Agent can use these tools as it sees fit.

While the examples we're constructing in this notebook are straightforward for brevity and simplicities sake - there is no limit to what you can build with Agents, as we'll see as we progress through the program.

So, let's begin by setting up our Tools!

You'll notice that we have to set up a function to allow our `OppenheimerInfo` tool to interface with the Agent - this is due to it have a specific required input. Creating custom tools is a pattern that you'll want to grow acustomed to as you use LangChain more and more.

In [29]:
from langchain.agents import Tool

def query_oppenheimer(input):
    return oppenheimer_multisource_chain.invoke({"question" : input})

tools = [
    Tool(
        name = "BarbieTool",
        func=barbie_retriever_agent_executor.invoke,
        description="Used for answering questions about the film Barbie."
    ),
    Tool(
        name = "OppenheimerTool",
        func=query_oppenheimer,
        description="Used for answering questions about the film Oppenheimer."
    ),
]

Now that we've set up our Agents toolbelt, let's set up the LLM that will be powering it!

I would suggest playing around with these prompts - and experiments to find what works best for you.

RELEVANT DOCS:
- [`ZeroShotAgent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.mrkl.base.ZeroShotAgent.html#langchain-agents-mrkl-base-zeroshotagent)

In [30]:
from langchain.agents import ZeroShotAgent, AgentExecutor

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "agent_scratchpad"]
)

In [31]:
from langchain import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt)

All that's left to do now is create our `ZeroShotAgent` and our `AgentExecutor`, which are the "reasoner" and "actor" halfs of the `ReAct` method of Agent implementation.

Read all about the `ReAct` framework [here](https://react-lm.github.io/)

In [32]:
barbenheimer_agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

barbenheimer_agent_chain = AgentExecutor.from_agent_and_tools(agent=barbenheimer_agent, tools=tools, verbose=True)

## Conclusion

All that is left to do now, is feed inputs to your Agent and watch it work!

Remember to use the `{"input" : "YOUR QUERY HERE"}` format when prompting the Agent.

In [33]:
barbenheimer_agent_chain.invoke({"input" : "What did people like about the Barbie movie?"})



> Entering new AgentExecutor chain...
Thought: I need to find out what people liked about the Barbie movie.
Action: BarbieTool
Action Input: "What did people like about the Barbie movie?"

> Entering new AgentExecutor chain...
People who enjoyed the Barbie movie appreciated various aspects of it. Here are some things that people liked about the movie:

1. Visual Spectacle: One reviewer mentioned that the movie is a wonderful visual spectacle. It likely includes visually stunning scenes and creative physical comedy.

2. Fun and Entertainment: Another reviewer mentioned that the movie is fun and entertaining for all ages. This suggests that the movie provides an enjoyable and engaging experience.

3. Doll-Like Elements: Some viewers appreciated the doll-like elements in the movie, such as the plastic and fake food, the gliding cars, and the doll-like movements of the actors. These elements may have added a unique and whimsical touch to the film.

4. Empowerment Themes: The movie seemed

{'input': 'What did people like about the Barbie movie?',
 'output': "People liked various aspects of the Barbie movie, including its visual spectacle, fun and entertainment value, doll-like elements, empowerment themes, and unique perspective on gender norms. However, it's important to note that these are individual opinions, and your own experience and interpretation of the movie may vary."}

In [34]:
barbenheimer_agent_chain.run({"input" : "What did people like about the Oppenheimer movie?"})



> Entering new AgentExecutor chain...
Thought: I don't have any information about the Oppenheimer movie.
Action: OppenheimerTool
Action Input: "What did people like about the Oppenheimer movie?"
Observation: content='People liked the screenplay, the performances of the cast (particularly those of Murphy and Downey), and the visuals of the Oppenheimer movie. They also praised the film for being compelling, engaging, and simple to understand. The cinematography was also highly praised, and the film received critical acclaim for its cast performances, screenplay, and visuals.' additional_kwargs={} example=False
Thought:I now know the final answer
Final Answer: People liked the screenplay, the performances of the cast (particularly those of Murphy and Downey), and the visuals of the Oppenheimer movie. They also praised the film for being compelling, engaging, and simple to understand. The cinematography was also highly praised, and the film received critical acclaim for its cast performa

'People liked the screenplay, the performances of the cast (particularly those of Murphy and Downey), and the visuals of the Oppenheimer movie. They also praised the film for being compelling, engaging, and simple to understand. The cinematography was also highly praised, and the film received critical acclaim for its cast performances, screenplay, and visuals.'

In [35]:
barbenheimer_agent_chain.run({"input" : "Did the movies Barbie and Oppenheimer share similar themes or ideas?"})



> Entering new AgentExecutor chain...
Thought: I need to determine if the movies Barbie and Oppenheimer share similar themes or ideas.
Action: OppenheimerTool
Action Input: "themes and ideas of Oppenheimer"
Observation: content="The themes and ideas explored in the Oppenheimer movie include:\n\n1. Ethical dilemmas: The movie delves into the ethical implications of Oppenheimer's involvement in the development of the atomic bomb. It raises questions about the responsibility of scientists and the consequences of their actions.\n\n2. Delayed reactions: The film explores the concept of delayed reactions, highlighting how individuals may not fully comprehend the impact of their actions in the moment. It suggests that the long-term consequences of Oppenheimer's work become apparent over time.\n\n3. Personal conflicts: Oppenheimer's internal conflicts and personal struggles are depicted in the movie. It delves into his psyche and the emotional toll of his choices, particularly in the face of

'The movies Oppenheimer and Barbie explore different themes and ideas. Oppenheimer delves into complex themes of ethics, personal conflicts, trust, and the consequences of scientific advancements. On the other hand, Barbie explores themes of self-discovery, empowerment, breaking gender stereotypes, individuality and acceptance, friendship and collaboration, and imagination and creativity. While both movies touch on personal conflicts and the consequences of actions, they have distinct focuses and explore different aspects of these themes.'

## Next Steps

It's time to build a Chainlit (or Gradio) application and host it on Hugging Face Spaces! :ship: